In [6]:
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
df = sns.load_dataset('tips')
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [8]:
# Since time is a nominal variable, we will use label encoder
encoder = LabelEncoder()
df['time']=encoder.fit_transform(df['time'])
df['time'].value_counts()

time
0    176
1     68
Name: count, dtype: int64

In [9]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [10]:

X=df.drop('time', axis=1)
y=df['time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [11]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [12]:
from sklearn.impute import SimpleImputer #Replace missing value in dataset >> Using mean, median, most_frequent values
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [13]:
num_col = ['total_bill','tip','size']
cat_col = ['sex','smoker', 'day']

In [14]:
num_pipline = Pipeline(steps=[('SimpleImputer',SimpleImputer(strategy='median')),('StandardScaler',StandardScaler())])
cat_pipline = Pipeline(steps=[('SimpleImputer',SimpleImputer(strategy='most_frequent')),('OneHotEncoder',OneHotEncoder())])

In [15]:
preprocessor = ColumnTransformer([('Num_pipline',num_pipline,num_col),('cat_pipline',cat_pipline,cat_col)])

In [16]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [26]:
models={
  "Logistic Regrassion":LogisticRegression(),
  "SVM":SVC(),
  "tree":DecisionTreeClassifier(),
  "GaussianNB":GaussianNB(),
  'RandomForest':RandomForestClassifier()
}

In [27]:
from sklearn.metrics import accuracy_score
def model_train_eval(X_train, X_test, y_train, y_test, models):
  evaluation ={}
  for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)
    model_accu =  accuracy_score(y_test, y_pred)
    evaluation[list(models.keys())[i]] = model_accu 
  return evaluation



In [28]:
model_train_eval(X_train, X_test, y_train, y_test, models)

{'Logistic Regrassion': 0.9183673469387755,
 'SVM': 0.9183673469387755,
 'tree': 0.9183673469387755,
 'GaussianNB': 0.9183673469387755,
 'RandomForest': 0.8979591836734694}

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
param ={
  'criterion': ['gini', 'entropy', 'log_loss'],
  'max_depth':[1, 2, 3, 4, 5, 10, None],
  'n_estimators':[50, 100, 200, 300]
}

In [22]:
from sklearn.model_selection import RandomizedSearchCV
Random_model = RandomizedSearchCV(rf,param_distributions=param, verbose=3, scoring='accuracy')

In [23]:
Random_model.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.923 total time=   0.3s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.974 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=1.000 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=1.000 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=0.923 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=0.974 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=1.000 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=1.000 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=50;, score=1.000 total time=  

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [1, 2, 3, 4, 5, 10, None],
                                        'n_estimators': [50, 100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [24]:
Random_model.best_params_

{'n_estimators': 200, 'max_depth': 5, 'criterion': 'entropy'}

In [25]:
Random_model.best_score_

0.9794871794871796